This notebook is just meant to test the codes before I compile them as .py files.

In [5]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from astropy.io import fits
import astropy.io.ascii as astropy_ascii
from astropy.table import Table

# 1. Fetch coordinates (ra, dec) from raw data

In [8]:
# 6dFGS
df = pd.read_csv('data/raw/6dfgs/campbell_table8.ascii', delim_whitespace=True)

# ra is still in hour -> need to convert to degrees
df_test = df[['ra', 'dec']]

In [10]:
Table.from_pandas(df_test).write('6dfgs_coord.ascii', format='ipac', overwrite=True)

In [23]:
# LAMOST
with fits.open('data/raw/lamost/lamost_DR7_VDcat_20200825.fits') as hdul:
    df = pd.DataFrame(hdul[1].data)

df[['ra', 'dec']].describe()

,ra,dec
count,85861.000000,85861.000000
mean,172.739314,23.402422
std,90.718450,17.346885
min,0.006891,-8.783024
25%,133.475564,7.475573
50%,171.662455,22.503952
75%,219.158605,36.137900
max,359.994162,77.790749


In [13]:
# # sdss
# df = pd.read_csv('data/raw/sdss/SDSS_spectro_mrafifrbbn.csv')

# df[['ra', 'dec']].describe()

# 2. Merge the spectroscopy and photometry

In [31]:
# Compare the original raw data to 2MASS (check if the order is the same)
with fits.open('data/raw/lamost/lamost_DR7_VDcat_20200825.fits') as hdul:
    df_lamost_spectro = pd.DataFrame(hdul[1].data)
    
df_lamost_tmass = pd.read_csv('data/raw/2mass/lamost_tmass.csv', low_memory=False)

df = pd.merge(df_lamost_spectro[['ra', 'dec']], df_lamost_tmass[['ra_01', 'dec_01']], left_index=True, right_index=True)

df['delta_ra'] = df['ra_01']-df['ra']
df['delta_dec'] = df['dec_01']-df['dec']


df[['delta_ra', 'delta_dec']].describe()

,delta_ra,delta_dec
count,8.586100e+04,85861.0
mean,1.164661e-14,0.0
std,2.621386e-11,0.0
min,-4.000015e-09,0.0
25%,0.000000e+00,0.0
50%,0.000000e+00,0.0
75%,0.000000e+00,0.0
max,5.000004e-09,0.0


In [33]:
# Compare the original raw data to 2MASS
df_6df_spectro = pd.read_csv('data/raw/6dfgs/campbell_table8.ascii', delim_whitespace=True)
df_6df_spectro['ra'] = df_6df_spectro['ra']*15

df_6df_tmass = pd.read_csv('data/raw/2mass/6dfgs_tmass.csv', low_memory=False)

df = pd.merge(df_6df_spectro[['ra', 'dec']], df_6df_tmass[['ra_01', 'dec_01']], left_index=True, right_index=True)

df['delta_ra'] = df['ra_01']-df['ra']
df['delta_dec'] = df['dec_01']-df['dec']

df[['delta_ra', 'delta_dec']].describe()

,delta_ra,delta_dec
count,1.110200e+04,11102.0
mean,2.142063e-16,0.0
std,1.650116e-14,0.0
min,-5.684342e-14,0.0
25%,0.000000e+00,0.0
50%,0.000000e+00,0.0
75%,0.000000e+00,0.0
max,5.684342e-14,0.0


## Main code for merging

In [34]:
with fits.open('data/raw/lamost/lamost_DR7_VDcat_20200825.fits') as hdul:
    df_lamost_spectro = pd.DataFrame(hdul[1].data)
    
df_lamost_tmass = pd.read_csv('data/raw/2mass/lamost_tmass.csv', low_memory=False)

df_lamost_tmass

# df = pd.merge(df_lamost_spectro[['ra', 'dec']], df_lamost_tmass[['ra_01', 'dec_01']], left_index=True, right_index=True)

# df['delta_ra'] = df['ra_01']-df['ra']
# df['delta_dec'] = df['dec_01']-df['dec']


# df[['delta_ra', 'delta_dec']].describe()

,cntr_01,dist_x,pang_x,ra_01,dec_01,designation,ra,dec,glon,glat,...,k_flg_10,j_m_20,j_msig_20,j_flg_20,h_m_20,h_msig_20,h_flg_20,k_m_20,k_msig_20,k_flg_20
0,1,NaN,NaN,331.169070,0.174930,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2,NaN,NaN,331.746950,0.543000,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3,NaN,NaN,331.813960,0.524550,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,4,NaN,NaN,334.111240,-0.902980,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,5,NaN,NaN,332.610020,1.613710,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
85856,85857,0.663243,-65.294939,226.245001,27.678836,15045875+2740440,226.244812,27.678913,41.824,60.447,...,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
85857,85858,NaN,NaN,226.953110,27.409455,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
85858,85859,0.116333,-81.098909,227.103277,27.522263,15082477+2731201,227.103241,27.522268,41.701,59.673,...,0.0,14.177,0.161,1.0,13.772,0.277,1.0,12.947,0.221,1.0
85859,85860,0.579073,-157.866464,226.632285,26.970316,15063173+2658126,226.632217,26.970167,40.499,60.014,...,0.0,14.459,0.212,0.0,13.776,0.280,0.0,13.910,0.543,0.0


In [42]:
tmass_response_columns = df_lamost_tmass.columns.tolist()
tmass_required_columns = tmass_response_columns[tmass_response_columns.index('designation'):]

df_lamost_tmass[tmass_required_columns]

,designation,ra,dec,glon,glat,j_ba,j_phi,h_ba,h_phi,k_ba,...,k_flg_10,j_m_20,j_msig_20,j_flg_20,h_m_20,h_msig_20,h_flg_20,k_m_20,k_msig_20,k_flg_20
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
85856,15045875+2740440,226.244812,27.678913,41.824,60.447,1.0,90.0,1.0,90.0,1.0,...,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
85857,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
85858,15082477+2731201,227.103241,27.522268,41.701,59.673,0.5,-65.0,0.5,-60.0,0.5,...,0.0,14.177,0.161,1.0,13.772,0.277,1.0,12.947,0.221,1.0
85859,15063173+2658126,226.632217,26.970167,40.499,60.014,0.4,-85.0,0.5,-80.0,0.5,...,0.0,14.459,0.212,0.0,13.776,0.280,0.0,13.910,0.543,0.0
